# emb_1

* validを2017/01/01~2017/09/30にする
* NNで回帰してみた

In [1]:
import optuna
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import timedelta, datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import pytorch_lightning as pl
from test_tube import Experiment
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import Trainer
from pytorch_lightning.logging import TestTubeLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

use_cuda = torch.cuda.is_available() and True
device = torch.device("cuda" if use_cuda else "cpu")

pd.set_option('display.max_columns', 500)
warnings.simplefilter("ignore")

### load dataset

In [2]:
DATA_DIR = '../../dataset/input/'
train_raw = pd.read_csv(DATA_DIR + 'train.csv')
test_raw = pd.read_csv(DATA_DIR + 'test.csv')

In [3]:
train_raw.head()

,holiday,temperature,rain_in_hour,snow_in_hour,clouds_cover,weather,weather_detail,timestamp,traffic_volume
0,New Years Day,263.49,0.0,0.0,58,Clouds,broken clouds,2013-01-01 00:00:00,1439
1,None,263.78,0.0,0.0,40,Clouds,scattered clouds,2013-01-01 01:00:00,1502
2,None,264.16,0.0,0.0,75,Snow,heavy snow,2013-01-01 02:00:00,933
3,None,263.95,0.0,0.0,90,Clouds,overcast clouds,2013-01-01 03:00:00,576
4,None,263.65,0.0,0.0,90,Clouds,overcast clouds,2013-01-01 04:00:00,372


In [4]:
test_raw.head()

,holiday,temperature,rain_in_hour,snow_in_hour,clouds_cover,weather,weather_detail,timestamp
0,New Years Day,249.36,0.0,0.0,1,Clear,sky is clear,2018-01-01 00:00:00
1,None,249.08,0.0,0.0,1,Clear,sky is clear,2018-01-01 01:00:00
2,None,248.86,0.0,0.0,1,Clear,sky is clear,2018-01-01 02:00:00
3,None,248.72,0.0,0.0,1,Clear,sky is clear,2018-01-01 03:00:00
4,None,248.43,0.0,0.0,1,Clear,sky is clear,2018-01-01 04:00:00


In [5]:
print("shape of train df : ", train_raw.shape)
print("shape of test df : ", test_raw.shape)

shape of train df :  (37696, 9)
shape of test df :  (7949, 8)


### preprocess

In [6]:
# remove irregular data
train_raw = train_raw[train_raw['temperature'] != 0]
train_raw = train_raw[train_raw['rain_in_hour']  != 9831.30]

In [7]:
train_raw.shape

(37685, 9)

In [8]:
n_train = len(train_raw)
tmp_df = pd.concat([train_raw, test_raw], sort=True)

In [9]:
# feature enginnering from timestamp
def preprocessor_for_date(df):
    # basic
    df['date'] = pd.to_datetime(df['timestamp'])
    df['Year'] = df['date'].apply(lambda x: x.year)
    df['Month'] = df['date'].apply(lambda x: x.month)
    df['Day'] = df['date'].apply(lambda x: x.day)
    df['Hour'] = df['date'].apply(lambda x: x.hour)
    df['DayOfWeek'] = df['date'].apply(lambda x: x.dayofweek)
    df['DayOfYear'] = df['date'].apply(lambda x: x.dayofyear)
    df['WeekOfYear'] = df['date'].apply(lambda x: x.weekofyear)
    df['WeekOfMonth'] = df['date'].apply(lambda x: x.day // 7 + 1)
    return df

tmp_df = preprocessor_for_date(tmp_df)

In [10]:
# fill correct value for holidays column
def preprocessor_for_holidays(df):
    df['date'] = pd.to_datetime(df['timestamp'])
    holidays_col = df[df['holiday'] != 'None']
    for holiday, date in zip(holidays_col['holiday'], holidays_col['date']):
        df['holiday'].mask((df['date'] > date) & (df['date'] < date +  timedelta(days=1)), holiday, inplace=True)
    return df

tmp_df = preprocessor_for_holidays(tmp_df)

In [11]:
# label encoding
def label_encoding(df, categorical_features):
    label_encoders = {}
    for cat_col in categorical_features:
        label_encoders[cat_col] = LabelEncoder()
        df[cat_col] = label_encoders[cat_col].fit_transform(df[cat_col])
    return df

categorical_features = ['holiday', 'weather', 'weather_detail', 'Month', 'Day', 'Hour', 
                        'DayOfWeek', 'DayOfYear', 'WeekOfYear', 'WeekOfMonth']
tmp_df = label_encoding(tmp_df, categorical_features)

# emb_dimsが必要
cat_dims = [int(tmp_df[col].nunique()) for col in categorical_features]
emb_dims = [(x, min(50, (x + 1) // 2)) for x in cat_dims]

In [12]:
# scaling 
def scaling_continuous_feature(df, numerical_features):
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])
    return df

continuous_feature = ['temperature', 'rain_in_hour', 'snow_in_hour', 'clouds_cover']
tmp_df = scaling_continuous_feature(tmp_df, continuous_feature)

In [13]:
tmp_df.head()

,clouds_cover,holiday,rain_in_hour,snow_in_hour,temperature,timestamp,traffic_volume,weather,weather_detail,date,Year,Month,Day,Hour,DayOfWeek,DayOfYear,WeekOfYear,WeekOfMonth
0,0.243744,6,-0.133546,-0.027985,-1.412115,2013-01-01 00:00:00,1439.0,1,2,2013-01-01 00:00:00,2013,0,0,0,1,0,0,0
1,-0.217067,6,-0.133546,-0.027985,-1.389521,2013-01-01 01:00:00,1502.0,1,24,2013-01-01 01:00:00,2013,0,0,1,1,0,0,0
2,0.678953,6,-0.133546,-0.027985,-1.359916,2013-01-01 02:00:00,933.0,8,10,2013-01-01 02:00:00,2013,0,0,2,1,0,0,0
3,1.062962,6,-0.133546,-0.027985,-1.376277,2013-01-01 03:00:00,576.0,1,19,2013-01-01 03:00:00,2013,0,0,3,1,0,0,0
4,1.062962,6,-0.133546,-0.027985,-1.399649,2013-01-01 04:00:00,372.0,1,19,2013-01-01 04:00:00,2013,0,0,4,1,0,0,0


In [14]:
tmp_df.tail()

,clouds_cover,holiday,rain_in_hour,snow_in_hour,temperature,timestamp,traffic_volume,weather,weather_detail,date,Year,Month,Day,Hour,DayOfWeek,DayOfYear,WeekOfYear,WeekOfMonth
7944,0.678953,7,-0.133546,-0.027985,0.142922,2018-09-30 19:00:00,NaN,1,2,2018-09-30 19:00:00,2018,8,29,19,6,272,38,4
7945,1.062962,7,-0.133546,-0.027985,0.089165,2018-09-30 20:00:00,NaN,1,19,2018-09-30 20:00:00,2018,8,29,20,6,272,38,4
7946,1.062962,7,-0.133546,-0.027985,0.086828,2018-09-30 21:00:00,NaN,10,21,2018-09-30 21:00:00,2018,8,29,21,6,272,38,4
7947,1.062962,7,-0.133546,-0.027985,0.036967,2018-09-30 22:00:00,NaN,1,19,2018-09-30 22:00:00,2018,8,29,22,6,272,38,4
7948,1.062962,7,-0.133546,-0.027985,0.039304,2018-09-30 23:00:00,NaN,1,19,2018-09-30 23:00:00,2018,8,29,23,6,272,38,4


In [15]:
train_tmp = tmp_df[:n_train]
valid = train_tmp[(train_tmp['date'] >= "2017-01-01") & (train_tmp['date'] < "2017-10-01")]
train = train_tmp[(train_tmp['date'] < "2017-01-01") | (train_tmp['date'] >= "2017-10-01")]
test = tmp_df[n_train:]
del test['traffic_volume']

print(train.shape, valid.shape, test.shape)

(29733, 18) (7952, 18) (7949, 17)


### Train

In [16]:
# define the custom dataset
class TrafficVolumePredictionDataset(Dataset):
    def __init__(self, data, cont_cols=None, output_col=None, unused_cols=None):
        self.num_data = data.shape[0]
        if output_col:
            raw_y = data[output_col].astype(np.float32).values.reshape(-1, 1)
            self.y = np.log(raw_y + 1)
        else:
            # for test inference
            self.y =  np.zeros((self.num_data, 1))
        self.cont_cols = cont_cols
        self.cat_cols = [col for col in data.columns 
                         if col not in self.cont_cols + [output_col] + unused_cols]
        self.cont_X = data[self.cont_cols].astype(np.float32).values
        self.cat_X = data[self.cat_cols].astype(np.int64).values
        
    def __len__(self):
        return self.num_data

    def __getitem__(self, idx):
        return [self.y[idx], self.cont_X[idx], self.cat_X[idx]]
    
# define custom loss
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss

# define my model
class MyModel(nn.Module):
    def __init__(self, emb_dims, num_of_cont, lin_layer_sizes):
        super(MyModel, self).__init__()
        
        # embeeding layers
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y) for x, y in emb_dims])

        # Linear Layers
        self.num_of_embs = sum([y for x, y in emb_dims])
        self.num_of_cont = num_of_cont
        first_lin_layer = nn.Linear(self.num_of_embs + self.num_of_cont, lin_layer_sizes[0])
        self.lin_layers = \
            nn.ModuleList([first_lin_layer] + \
            [nn.Linear(lin_layer_sizes[i], lin_layer_sizes[i + 1])
            for i in range(len(lin_layer_sizes) - 1)])
        
        # initialize weight
        for lin_layer in self.lin_layers:
            nn.init.kaiming_normal_(lin_layer.weight.data)

        # Output Layer
        self.output_layer = nn.Linear(lin_layer_sizes[-1], 1)
        nn.init.kaiming_normal_(self.output_layer.weight.data)

        # Batch Norm Layers
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(size) for size in lin_layer_sizes])
        
    def forward(self, cont_data, cat_data):
        # embedding
        x = [emb_layer(cat_data[:, i]) for i, emb_layer in enumerate(self.emb_layers)]
        x = torch.cat(x, 1)

        # concat
        x = torch.cat([x, cont_data], 1) 
        
        # linear
        for lin_layer, bn_layer in zip(self.lin_layers, self.bn_layers):
            x = F.relu(lin_layer(x))
            x = bn_layer(x)

        x = self.output_layer(x)

        return x

In [17]:
## define pytorch lightnig class

class TrafficVolumePrediction(pl.LightningModule):
    def __init__(self, emb_dims, num_of_cont, lin_layer_sizes):
        super(TrafficVolumePrediction, self).__init__()
        # embbeding layer
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y) for x, y in emb_dims])

        # Linear Layers
        self.num_of_embs = sum([y for x, y in emb_dims])
        self.num_of_cont = num_of_cont
        first_lin_layer = nn.Linear(self.num_of_embs + self.num_of_cont, lin_layer_sizes[0])
        self.lin_layers = \
            nn.ModuleList([first_lin_layer] + \
            [nn.Linear(lin_layer_sizes[i], lin_layer_sizes[i + 1])
            for i in range(len(lin_layer_sizes) - 1)])
        
        # initialize weight
        for lin_layer in self.lin_layers:
            nn.init.kaiming_normal_(lin_layer.weight.data)

        # Output Layer
        self.output_layer = nn.Linear(lin_layer_sizes[-1], 1)
        nn.init.kaiming_normal_(self.output_layer.weight.data)

        # Batch Norm Layers
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(size) for size in lin_layer_sizes])
        # loss function
        self.criterion = RMSELoss()
    
    def forward(self, cont_data, cat_data):
        # embedding
        x = [emb_layer(cat_data[:, i]) for i, emb_layer in enumerate(self.emb_layers)]
        x = torch.cat(x, 1)

        # concat
        x = torch.cat([x, cont_data], 1) 
        
        # linear
        for lin_layer, bn_layer in zip(self.lin_layers, self.bn_layers):
            x = F.relu(lin_layer(x))
            x = bn_layer(x)

        x = self.output_layer(x)
        return x
    
    def training_step(self, batch, batch_nb):
        # REQUIRED
        y, cont_x, cat_x = batch
        y_hat = self.forward(cont_x, cat_x)
        return {'loss': self.criterion(y_hat, y)}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        y, cont_x, cat_x = batch
        y_hat = self.forward(cont_x, cat_x)
        return {'val_loss': self.criterion(y_hat, y)}

    def validation_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print('val_loss ', avg_loss)
        return {'avg_val_loss': avg_loss}
    
    def test_step(self, batch, batch_nb):
        # OPTIONAL
        y, cont_x, cat_x = batch
        y_hat = self.forward(cont_x, cat_x)
        return {'val_loss': self.criterion(y_hat, y)}

    def test_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        return {'avg_val_loss': avg_loss}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        return torch.optim.Adam(self.parameters(), lr=0.001)

    @pl.data_loader
    def tng_dataloader(self):
        # REQUIRED
        dataset = TrafficVolumePredictionDataset(
            data=train,
            cont_cols=['temperature', 'rain_in_hour', 'clouds_cover'],
            output_col='traffic_volume',
            unused_cols=['timestamp', 'Year', 'snow_in_hour', 'date']
        )
        return DataLoader(dataset, batch_size=256, shuffle=True)
    
    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        dataset = TrafficVolumePredictionDataset(
            data=valid,
            cont_cols=['temperature', 'rain_in_hour', 'clouds_cover'],
            output_col='traffic_volume',
            unused_cols=['timestamp', 'Year', 'snow_in_hour', 'date']
        )
        return DataLoader(dataset, batch_size=256, shuffle=True)

In [18]:
# define variables
emb_dims = emb_dims
cont_col = ['temperature', 'rain_in_hour', 'clouds_cover']
num_of_cont = len(cont_col)
lin_layer_sizes = [128, 64, 32]

In [19]:
# train
# pytorch lightningは、validデータを与えるとデフォルトでearly stoppingする

# custom checkpoint and early stop
checkpoint_callback = ModelCheckpoint(
    filepath='result/best_model/',
    save_best_only=True,
    verbose=True,
    monitor='avg_val_loss',
    mode='min'
)

model = TrafficVolumePrediction(emb_dims, num_of_cont, lin_layer_sizes)
trainer = Trainer(default_save_path='result', max_nb_epochs=100, 
                  checkpoint_callback=checkpoint_callback)
trainer.fit(model)

Validation sanity check:   0%|          | 0/5 [00:00<?, ?batch/s]

val_loss  tensor(0.3091)


Epoch 12:  98%|█████████▊| 146/149 [00:03<00:00, 60.72batch/s, batch_nb=116, loss=0.234, v_nb=2]

val_loss  tensor(0.2912)


Epoch 13:  96%|█████████▌| 143/149 [00:02<00:00, 67.91batch/s, batch_nb=116, loss=0.226, v_nb=2]

...

val_loss  tensor(0.2430)


Validating:  78%|███████▊  | 25/32 [00:00<00:00, 121.92batch/s]

val_loss  tensor(0.2431)


Epoch 102: 100%|██████████| 149/149 [00:03<00:00, 41.17batch/s, batch_nb=116, loss=0.124, v_nb=2]


1

## Predict

In [21]:
def predict(model, test_data_loader, checkpoint):
    model.load_state_dict(checkpoint['state_dict'])

    for param in model.parameters():
        param.requires_grad = False
        
    model.eval()
    test_preds = []
    for i, (y, cont_x, cat_x) in enumerate(tqdm(test_data_loader)):
        with torch.no_grad():
            cat_x = cat_x.to(device)
            cont_x = cont_x.to(device)
            preds = model.forward(cont_x, cat_x)
            test_preds.extend(preds.data.cpu().numpy())
    return test_preds


batch_size = 256
all_test_preds = []
checkpoint = torch.load('result/best_model/_ckpt_epoch_99.ckpt')
test_dataset = TrafficVolumePredictionDataset(
    data=test,
    cont_cols=['temperature', 'rain_in_hour', 'clouds_cover'],
    output_col=None,
    unused_cols=['timestamp', 'Year', 'snow_in_hour', 'date']
)

test_data_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size,
    shuffle=False
)

y_test_log_pred = predict(model, test_data_loader, checkpoint)

100%|██████████| 32/32 [00:00<00:00, 123.72it/s]


In [24]:
y_test_pred = np.exp(y_test_log_pred) - 1

#save predictions in rows
save_path = './submission/embv1lgb{}.txt'.format(9263289631)
np.savetxt(save_path, y_test_pred)